In [ ]:
%pip install transformers datasets
%pip install tensorflow torch
%pip install tf-keras
%pip install datasets
%pip install transformers
%pip install peft==0.10.0
%pip install -U huggingface_hub
%pip install accelerate>=0.26.0
%pip install transformers

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# GPT-2 토크나이저 불러오기
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # 패딩 토큰 설정

/Users/won3956/Desktop/DSBA_project1/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0514 11:17:59.118000 5477 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [ ]:
from datasets import Dataset
import json

# JSON 파일 읽기 (파일 경로는 네가 사용하는 파일명으로 바꿔줘)
with open('training.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Hugging Face Dataset 형식으로 변환
dataset = Dataset.from_dict({
    "text": [
        f"### 입력:\n이름: {item['name']}\n" +
        "\n".join([f"{entry['date']}: {entry['score']}점" for entry in item["score_changes"]]) +
        f"\n\n### 출력:\n{item['output']}"
        for item in data
    ]
})


In [4]:
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

dataset = dataset.train_test_split(test_size=0.1)
train_tokenized = dataset["train"].map(tokenize, batched=True)
eval_tokenized = dataset["test"].map(tokenize, batched=True)

Map: 100%|██████████| 50/50 [00:00<00:00, 313.08 examples/s]


In [8]:
from transformers import DataCollatorForLanguageModeling

# 데이터 collator 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # GPT는 MLM이 아님
)


In [9]:
# 모델 불러오기
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # pad_token 추가 반영

Embedding(50257, 768)

In [10]:
from transformers import TrainingArguments

# trainer 훈련 설정
training_args = TrainingArguments(
    output_dir="./gpt2-dementia",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_steps=200,
    save_steps=500,
    warmup_steps=100,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    eval_strategy="epoch",
    report_to="none"
)

In [11]:
from transformers import Trainer
import accelerate

# Trainer 구성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=eval_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/var/folders/s3/6dlfc37j6vscfkx18rs0sw6m0000gn/T/ipykernel_5477/945720683.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
# train 시작
trainer.train()

/Users/won3956/Desktop/DSBA_project1/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,0.098600,0.058262
2,0.056100,0.047915
3,0.051800,0.045429


/Users/won3956/Desktop/DSBA_project1/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=675, training_loss=0.24846972094641792, metrics={'train_runtime': 921.4586, 'train_samples_per_second': 1.465, 'train_steps_per_second': 0.733, 'total_flos': 352744243200000.0, 'train_loss': 0.24846972094641792, 'epoch': 3.0})